Removed duplicate constrain from ll1 ll2 mdl and vehicle no

In [2]:
!pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------  2.1/2.1 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 9.7 MB/s  0:00:00

   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchemy]
   -------------------------- ------------- 2/3 [sqlalchem

Below is for keeping only 1st entry for duplicate record

In [4]:
import pandas as pd
import sqlite3
from datetime import datetime
from sqlalchemy import create_engine, text

# --- Load CSVs ---
mst_jobs = pd.read_csv("./29092025_Latest_DB/csv/mst_job.csv")
tbl_learnlicdet = pd.read_csv("./29092025_Latest_DB/csv/tbl_learnlicdet.csv")
tbl_permanentlicdet = pd.read_csv("./29092025_Latest_DB/csv/tbl_permanentlicdet.csv")
tbl_stufees = pd.read_csv("./29092025_Latest_DB/csv/tbl_stufees.csv")
yash_students = pd.read_csv("./29092025_Latest_DB/csv/yash_students.csv")

# --- Lowercase text values only ---
def lowercase_df(df):
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].str.lower().str.strip()
    return df

yash_students = lowercase_df(yash_students)

# --- Transform columns ---
yash_students[['ll_no_1', 'll_no_2']] = yash_students['learn_lic_no'].str.split(pat=',', n=1, expand=True)
yash_students[['ll_class_1', 'll_class_2']] = yash_students['learn_lic_type'].str.split(pat=',', n=1, expand=True)

yash_students.rename(columns={
    "StuID": "id",
    "StuPhone": "mobile_number",
    "StuName": "customer_name",
    "StuDOB": "customer_dob",
    "StuPerAdd": "address",
    "CarID": "vehicle_id",
    "InstID": "instructor_id",
    "learn_dt_issue": "ll_issued_date",
    "learn_dt_expiry": "ll_validity_date",
    "perm_lic_no": "mdl_no",
    "perm_lic_type": "mdl_class",
    "perm_dt_issue": "mdl_issued_date",
    "perm_dt_expiry": "mdl_validity_date",
    "Endorse": "endorsement",
    "EndorseDate": "endorsement_date",
    "StuSDWOf": "relation_name",
    "LastUpdatedDate": "created_on"
}, inplace=True)

# --- Add missing DB columns ---
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
yash_students["updated_on"] = now
yash_students["customer_image"] = None
yash_students["customer_signature"] = None
yash_students["endorsement_validity_date"] = None
yash_students["customer_vehicle_no"] = None

# --- Fix dates (invalid → None, not "NaT") ---
date_cols = [
    'customer_dob', 'll_issued_date', 'll_validity_date',
    'mdl_issued_date', 'mdl_validity_date',
    'endorsement_date', 'created_on'
]
for col in date_cols:
    if col in yash_students.columns:
        yash_students[col] = pd.to_datetime(yash_students[col], errors='coerce')
        yash_students[col] = yash_students[col].dt.strftime("%Y-%m-%d %H:%M:%S")
        yash_students[col] = yash_students[col].where(pd.notnull(yash_students[col]), None)

# --- Insert/Update into SQLite ---
database = "./29092025_Latest_DB/guide-mts-database.sqlite3"
engine = create_engine(f"sqlite:///{database}")

# Get table columns
with engine.begin() as conn:
    res = conn.execute(text("PRAGMA table_info(customers)")).fetchall()
    table_columns = [row[1] for row in res]

# Match dataframe to DB columns
columns_to_insert = [col for col in table_columns if col in yash_students.columns]
df = yash_students[columns_to_insert].copy()

# Remove rows with duplicate mobile_number except for the first occurrence
df = df.drop_duplicates(subset=['mobile_number'], keep='first')

# UPSERT (insert or update on id conflict)
with engine.begin() as conn:
    for _, row in df.iterrows():
        data = row.to_dict()
        cols = ", ".join(data.keys())
        placeholders = ", ".join([f":{c}" for c in data.keys()])
        updates = ", ".join([f"{c}=excluded.{c}" for c in data.keys() if c != "id"])

        sql = text(f"""
            INSERT INTO customers ({cols})
            VALUES ({placeholders})
            ON CONFLICT(id) DO UPDATE SET {updates};
        """)
        conn.execute(sql, data)

print("✅ Data migrated with upsert (insert/update) successfully!")


C:\Users\balot\AppData\Local\Temp\ipykernel_18108\2048895274.py:61: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  yash_students[col] = pd.to_datetime(yash_students[col], errors='coerce')


✅ Data migrated with upsert (insert/update) successfully!


below is for merging dupluicate entry data

In [1]:
import pandas as pd
import sqlite3
from datetime import datetime
from sqlalchemy import create_engine, text

# --- Load CSVs ---
mst_jobs = pd.read_csv("./29092025_Latest_DB/csv/mst_job.csv")
tbl_learnlicdet = pd.read_csv("./29092025_Latest_DB/csv/tbl_learnlicdet.csv")
tbl_permanentlicdet = pd.read_csv("./29092025_Latest_DB/csv/tbl_permanentlicdet.csv")
tbl_stufees = pd.read_csv("./29092025_Latest_DB/csv/tbl_stufees.csv")
yash_students = pd.read_csv("./29092025_Latest_DB/csv/yash_students.csv")

# --- Lowercase text values only ---
def lowercase_df(df):
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].str.lower().str.strip()
    return df

yash_students = lowercase_df(yash_students)

# --- Transform columns ---
yash_students[['ll_no_1', 'll_no_2']] = yash_students['learn_lic_no'].str.split(pat=',', n=1, expand=True)
yash_students[['ll_class_1', 'll_class_2']] = yash_students['learn_lic_type'].str.split(pat=',', n=1, expand=True)

yash_students.rename(columns={
    "StuID": "id",
    "StuPhone": "mobile_number",
    "StuName": "customer_name",
    "StuDOB": "customer_dob",
    "StuPerAdd": "address",
    "CarID": "vehicle_id",
    "InstID": "instructor_id",
    "learn_dt_issue": "ll_issued_date",
    "learn_dt_expiry": "ll_validity_date",
    "perm_lic_no": "mdl_no",
    "perm_lic_type": "mdl_class",
    "perm_dt_issue": "mdl_issued_date",
    "perm_dt_expiry": "mdl_validity_date",
    "Endorse": "endorsement",
    "EndorseDate": "endorsement_date",
    "StuSDWOf": "relation_name",
    "LastUpdatedDate": "created_on"
}, inplace=True)

# --- Add missing DB columns ---
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
yash_students["updated_on"] = now
yash_students["customer_image"] = None
yash_students["customer_signature"] = None
yash_students["endorsement_validity_date"] = None
yash_students["customer_vehicle_no"] = None

# --- Fix dates (invalid → None, not "NaT") ---
date_cols = [
    'customer_dob', 'll_issued_date', 'll_validity_date',
    'mdl_issued_date', 'mdl_validity_date',
    'endorsement_date', 'created_on'
]

for col in date_cols:
    if col in yash_students.columns:
        yash_students[col] = pd.to_datetime(yash_students[col], errors='coerce')
        yash_students[col] = yash_students[col].dt.strftime("%Y-%m-%d %H:%M:%S")
        yash_students[col] = yash_students[col].where(pd.notnull(yash_students[col]), None)

# DB path
database = "./29092025_Latest_DB/guide-mts-database.sqlite3"
engine = create_engine(f"sqlite:///{database}")

# Assume yash_students dataframe already prepared & cleaned as in previous script

# Get table columns
with engine.begin() as conn:
    res = conn.execute(text("PRAGMA table_info(customers)")).fetchall()
    table_columns = [row[1] for row in res]

columns_to_insert = [col for col in table_columns if col in yash_students.columns]
df = yash_students[columns_to_insert].copy()

# Build UPSERT with COALESCE (keep old value if new is NULL)
with engine.begin() as conn:
    for _, row in df.iterrows():
        data = row.dropna(how="all").to_dict()   # drop rows with all NaNs/None
        if not data:
            continue

        cols = ", ".join(data.keys())
        placeholders = ", ".join([f":{c}" for c in data.keys()])

        update_cols = [c for c in data.keys() if c != "mobile_number"]
        if not update_cols:
            continue  # nothing to update

        updates = ", ".join([
            f"{c} = COALESCE(excluded.{c}, customers.{c})"
            for c in update_cols
        ])

        sql = text(f"""
            INSERT INTO customers ({cols})
            VALUES ({placeholders})
            ON CONFLICT(mobile_number) DO UPDATE SET {updates};
        """)
        conn.execute(sql, data)


print("✅ Data merged successfully with NULL-safe updates!")


C:\Users\balot\AppData\Local\Temp\ipykernel_13564\731562474.py:62: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  yash_students[col] = pd.to_datetime(yash_students[col], errors='coerce')


✅ Data merged successfully with NULL-safe updates!
